In [23]:
%load_ext jupyter_black

# Set-up OpenAI API key

https://platform.openai.com/account/api-keys

In [24]:
import openai

with open("../API_KEY", "r") as f:
    key = f.read()

openai.api_key = key

# General Plan:
- Basic example of new `function_call`
- Converting existing functions for use with `function_call`
- Give GPT the ability to make new functions
- Allow GPT to use it's own newly created functions

    

In [25]:
# Main imports
import openai
import json  # Converting between string and dict

# Other imports that will be explained as they come up
import os
import inspect
import importlib.util
from typing import Callable

# Basic Example of `function_call`

## Example function for GPT to use

In [26]:
def get_sports_headlines(
    sport: str, team_names: list[str] = None, num_results: int = 3
) -> list[str]:
    """
    Gets the latest headlines for the given sport. Optionally filtering by specific team names

    Args:
        sport: Which sport to search within. Can be 'soccer', 'basketball', or 'baseball'.
        team_names: Optionally provide specific team names to search for
        num_results: How many headlines to return
    """
    # Here you would implement searching through a database or calls to a search API etc
    # We'll just use some fictional results I pre-prepared
    with open("fictional_headlines.json", "r") as f:
        all_headlines = json.load(f)
    headlines = all_headlines.get(sport, None)
    if not headlines:
        return "No headlines found"
    if team_names:
        headlines = [
            headline
            for headline in headlines
            if any([team in headline for team in team_names])
        ]
    return headlines[:num_results]

In [27]:
GET_SPORTS_HEADLINES_DESCRIPTION = {
    "name": "get_sports_headlines",
    "description": "Gets the latest headlines for the given sport. Optionally filtering by specific team names",
    "parameters": {
        "type": "object",
        "properties": {
            "sport": {
                "type": "string",
                "enum": ["soccer", "basketball", "baseball"],
                "description": "Which sport to search within",
            },
            "team_names": {
                "type": "array",
                "items": {"type": "string"},
                "description": "Optionally provide specific team names to search for",
            },
            "num_results": {
                "type": "number",
                "description": "How many headlines to return",
            },
        },
        "required": ["sport"],
    },
}

## Call to openai with new completion parameters
`functions` and `function_call`

In [28]:
user_message = "Based on the latest headlines for Manchester and Aresnal, can you tell me whether the teams won or lost for each headline?"
user_message

'Based on the latest headlines for Manchester and Aresnal, can you tell me whether the teams won or lost for each headline?'

In [29]:
# Let's see what we get back without using functions first
baseline_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[{"role": "user", "content": user_message}],
)
baseline_response["choices"][0]["message"]["content"]

"Sorry, as an AI language model, I don't have access to the latest headlines or live update sports scores. Please browse the internet or news outlets to get the latest information on Manchester and Arsenal Football Club's performance and scores."

---
As expected, not a very satisfying response

Now let's use the new model with the new API parameters

In [30]:
first_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": user_message}],
    functions=[
        GET_SPORTS_HEADLINES_DESCRIPTION,
    ],
    function_call="auto",
)
first_response

<OpenAIObject chat.completion id=chatcmpl-7Sw2W4Vck8ptS8p6YZWQ7SbOnOLfG at 0x19e75bb2c90> JSON: {
  "id": "chatcmpl-7Sw2W4Vck8ptS8p6YZWQ7SbOnOLfG",
  "object": "chat.completion",
  "created": 1687130644,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_sports_headlines",
          "arguments": "{\n  \"sport\": \"soccer\",\n  \"team_names\": [\"Manchester\", \"Arsenal\"],\n  \"num_results\": 5\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 130,
    "completion_tokens": 39,
    "total_tokens": 169
  }
}

---
We can see that GPT decided to make a `function_call` (rather than returning `content` like it would have before the `0613` update)

It even realised that we are probably talking about 'soccer' if we are asking about 'Manchester' and 'Arsenal'

## Get the response from the function

In [31]:
# Extract the returned message
function_call_message = first_response["choices"][0]["message"]

# From that message, get the name of the function called
function_name = function_call_message["function_call"]["name"]

# Also get the arguments (this is a string representation of a JSON dict of arguments)
arguments = function_call_message["function_call"].get("arguments")
# Convert from string to dict
arguments = json.loads(arguments)

# Make the call to the function with the arguments that the LLM decided
function_response = get_sports_headlines(
    sport=arguments["sport"],
    team_names=arguments.get("team_names", None),
    num_results=arguments.get("num_results", 5),
)
# function_response = get_weather_report(
#     **arguments,  # Unpack the arguments as keyword: value pairs
# )

# Look at what the function returned
print(f"Function output:\n{function_response}")

Function output:
['Manchester United Stuns Rivals: Late Goal Seals Victory in High-Stakes Match Against Liverpool', "Manchester United Triumphs: Bruno Fernandes' Hat Trick Silences Critics in Decisive Win over Chelsea", 'Arsenal Secures Victory: Pierre-Emerick Aubameyang Shines in Nail-biting Finish Against Tottenham', 'Manchester City Stunned: Late Own Goal Gives Aston Villa Shock Win', 'The Manchester Derby Ends in Stalemate: Defensive Masterclass from Both United and City']


## Convert response into message that GPT understands

In [32]:
function_response_message = {
    "role": "function",
    "name": function_name,
    "content": str(function_response),  # content must always be a string
}
function_response_message

{'role': 'function',
 'name': 'get_sports_headlines',
 'content': '[\'Manchester United Stuns Rivals: Late Goal Seals Victory in High-Stakes Match Against Liverpool\', "Manchester United Triumphs: Bruno Fernandes\' Hat Trick Silences Critics in Decisive Win over Chelsea", \'Arsenal Secures Victory: Pierre-Emerick Aubameyang Shines in Nail-biting Finish Against Tottenham\', \'Manchester City Stunned: Late Own Goal Gives Aston Villa Shock Win\', \'The Manchester Derby Ends in Stalemate: Defensive Masterclass from Both United and City\']'}

## Call to openai including the extra info

In [33]:
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": user_message},
        # Add the context of the previous function call and response
        function_call_message,
        function_response_message,
    ],
    # Leaving these here just to demonstrate that GPT only uses functions when necessary
    functions=[
        GET_SPORTS_HEADLINES_DESCRIPTION,
    ],
    function_call="auto",
)
second_response

<OpenAIObject chat.completion id=chatcmpl-7Sw2XpbFY0WpbWwCwUhUKlZ1bdiPJ at 0x19e75bb2f30> JSON: {
  "id": "chatcmpl-7Sw2XpbFY0WpbWwCwUhUKlZ1bdiPJ",
  "object": "chat.completion",
  "created": 1687130645,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Based on the latest headlines:\n\n1. Manchester United Stuns Rivals: Late Goal Seals Victory in High-Stakes Match Against Liverpool - Manchester United won.\n2. Manchester United Triumphs: Bruno Fernandes' Hat Trick Silences Critics in Decisive Win over Chelsea - Manchester United won.\n3. Arsenal Secures Victory: Pierre-Emerick Aubameyang Shines in Nail-biting Finish Against Tottenham - Arsenal won.\n4. Manchester City Stunned: Late Own Goal Gives Aston Villa Shock Win - Manchester City lost.\n5. The Manchester Derby Ends in Stalemate: Defensive Masterclass from Both United and City - The match ended in a draw."
      },
      "finish_reason": "st

In [34]:
print(second_response["choices"][0]["message"]["content"])

Based on the latest headlines:

1. Manchester United Stuns Rivals: Late Goal Seals Victory in High-Stakes Match Against Liverpool - Manchester United won.
2. Manchester United Triumphs: Bruno Fernandes' Hat Trick Silences Critics in Decisive Win over Chelsea - Manchester United won.
3. Arsenal Secures Victory: Pierre-Emerick Aubameyang Shines in Nail-biting Finish Against Tottenham - Arsenal won.
4. Manchester City Stunned: Late Own Goal Gives Aston Villa Shock Win - Manchester City lost.
5. The Manchester Derby Ends in Stalemate: Defensive Masterclass from Both United and City - The match ended in a draw.


---
Now GPT gives us a nice summary of that information, including an inferred win/loss based on the headline!

# Automate making the JSON description of functions for use with GPT

It's a bit tedious to write the JSON description of your functions, so let's automate that.

We'll get GPT to do the conversion for us. It won't be 100% reliable, but it will handle a wide variety of existing code

In [35]:
# Quickly demonstrate the use of these imports for those that aren't familiar
import inspect
from typing import Callable

print(
    "Inspect.getsource(get_weather_report):\n", inspect.getsource(get_sports_headlines)
)
print("---------------------")
print(
    "\nCheck functions are of type `Callable`:\n",
    isinstance(get_sports_headlines, Callable),
)

Inspect.getsource(get_weather_report):
 def get_sports_headlines(
    sport: str, team_names: list[str] = None, num_results: int = 3
) -> list[str]:
    """
    Gets the latest headlines for the given sport. Optionally filtering by specific team names

    Args:
        sport: Which sport to search within. Can be 'soccer', 'basketball', or 'baseball'.
        team_names: Optionally provide specific team names to search for
        num_results: How many headlines to return
    """
    # Here you would implement searching through a database or calls to a search API etc
    # We'll just use some fictional results I pre-prepared
    with open("fictional_headlines.json", "r") as f:
        all_headlines = json.load(f)
    headlines = all_headlines.get(sport, None)
    if not headlines:
        return "No headlines found"
    if team_names:
        headlines = [
            headline
            for headline in headlines
            if any([team in headline for team in team_names])
        ]


## Build the system prompt

This will help guide GPT to do what we want it to do

In [36]:
SYSTEM_PROMPT_JSON_REP = '''Your job is to convert a python function into a json representation with a specific form.
For example, given this function:
```
def get_weather_report(day_of_week: int, weather_type: str, temperature: float = 10.0) -> str:
    """
    Converts information about weather into a string representation.

    Args:
        day_of_week (int): The day of the week from 0 to 6.
        weather_type (str): The type of weather, can be "sunny", "rainy", or "windy".
        temperature (float, optional): Temperature in Celsius. Defaults to 10.0.

    Returns:
        str: A string representation of the weather report.
    """
    return f'For the {day_of_week}th day of the week, the weather is predicted to be {weather_type} with a max temperature of {temperature}'
```

You should return:
```
{
    "name": "get_weather_report",
    "description": "Converts information about weather into a string representation",
    "parameters": {
        "type": "object",
        "properties": {
            "day_of_week": {
                "type": "number",
                "description": "The day of the week from 0 to 6",
            },
            "weather_type": {"type": "string", "enum": ["sunny", "rainy", "windy"]},
            "temperature": {"type": "number", "description": "Temperature in Celsius. Defaults to 10.0."},
        },
        "required": ["day_of_week", "weather_type"],
    },
}
```
Return the JSON ONLY.'''

## Make the function

Make it easy to get the JSON representation directly from an existing function

In [37]:
def get_json_representation(func: Callable) -> dict:
    """
    Uses the openai.ChatCompletion.create endpoint to return the JSON representation of a function (for GPT to use)

    Args:
        function_str (str): The function to generate a JSON representation for

    Returns:
        dict: The JSON representation of func
    """
    # Get the code of the given func
    function_code = inspect.getsource(func)

    # Ask OpenAI to make the conversion to JSON format
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        temperature=0.0,  # Deterministic output (most probable next word every time)
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT_JSON_REP},
            {"role": "user", "content": function_code},  # Then give the func code
        ],
    )

    # Extract the assistant's response
    assistant_response = response["choices"][0]["message"]["content"]

    # convert to dict (JSON)
    description_json = json.loads(assistant_response)
    return description_json

In [38]:
get_json_representation(get_sports_headlines)

{'name': 'get_sports_headlines',
 'description': 'Gets the latest headlines for the given sport. Optionally filtering by specific team names',
 'parameters': {'type': 'object',
  'properties': {'sport': {'type': 'string',
    'enum': ['soccer', 'basketball', 'baseball']},
   'team_names': {'type': 'array',
    'items': {'type': 'string'},
    'description': 'Optionally provide specific team names to search for'},
   'num_results': {'type': 'number',
    'description': 'How many headlines to return'}},
  'required': ['sport']},
 'returns': {'type': 'array', 'items': {'type': 'string'}}}

---
That's pretty good. GPT added a `returns` field here that we didn't ask for, but that's OK

# Get GPT to make new functions

Let's guide GPT on how to make functions for itself

## Create some necessary helper functions

To actually get a useable function from the string GPT will return to us, we'll save the text to a `.py` file, then load it using `importlib`

These are the functions required to do that

In [40]:
import os
import importlib.util

FUNCTIONS_FOLDER = "functions"
os.makedirs(FUNCTIONS_FOLDER, exist_ok=True)


def sanitize_python_code(code: str) -> str:
    """If the code is surrounded by markdown triple backticks, they are removed."""
    lines = code.split("\n")
    filtered_lines = [line for line in lines if not line.startswith("```")]
    filtered_code = "\n".join(filtered_lines)
    return filtered_code


def filepath(folder: str, function_name: str) -> str:
    return os.path.join(folder, f"{function_name}.py")


def write_to_py_file(folder: str, function_name: str, file_contents: str):
    """Write code to .py file"""
    # Save function to .py file
    file_contents = sanitize_python_code(file_contents)
    with open(filepath(folder, function_name), "w") as f:
        f.write(file_contents)


def load_function_from_file(folder: str, function_name: str) -> Callable:
    """
    Loads a function from a .py file.
    """
    # Load the spec of the module
    spec = importlib.util.spec_from_file_location(
        "temporary", filepath(folder, function_name)
    )

    # Create a module from the spec
    module = importlib.util.module_from_spec(spec)

    # Execute the module to get the function
    spec.loader.exec_module(module)

    # Get the function from the module
    function = getattr(module, function_name)

    return function

## Build the system prompt

In [87]:
SYSTEM_PROMPT_MAKE_NEW_FUNCTION = """You are an expert python coder that will be tasked with generating the code to go in a .py file for a single function given some specific information from the user.
You will be provided:
    - function_name: The name to give the new function
    - arg_descriptions: Descriptions of all the arguments the function should take (if their types are missing, try to infer them)
    - description: What the function should do with the given arguments

When generating the new function you should follow these rules:
    - Include ONLY the text that will be in the python file (e.g. starting with `import ...` unless no imports are necessary in which case, starting with `def ...`)
    - Do NOT include any plain text explanation at the end of the written code
    - Use the latest python programming techniques and best practices
    - Use the latest/best python libraries when appropriate (e.g. if plotting, use `plotly` instead of `matplotlib` because plotly is better library even though matplotlib is better known)
    - Always include a google style docstring
    - Include type hints for the inputs and output
    - Include all necessary imports (including `typing` ones e.g. List)
"""

## Make the `make_new_function` function

In [88]:
def make_new_function(
    function_name: str, arg_descriptions: str, description: str
) -> Callable:
    """
    Use this if an existing function doesn't exist, and it would be helpful to have a new function to complete a task.
    The new function will be made to carry out the task described in the `description` given the arguments described by `arg_descriptions`

    Args:
        function_name: Name to give the new function (should follow python naming conventions)
        arg_descriptions: A description of any arguments that the function should take (including type, and default value if appropriate)
        description: A description of what the function should do (including the what it should output)
    """

    # Format the message we'll send to GPT in the form we have described in the system prompt
    formatted_input = f"function_name: {function_name}\narg_descriptions: {arg_descriptions}\ndescription: {description}"

    # Ask OpenAI to make the new function
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        temperature=0.0,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT_MAKE_NEW_FUNCTION},
            {"role": "user", "content": formatted_input},
        ],
    )

    # Extract the assistant's response
    function_code = response["choices"][0]["message"]["content"]

    # Tidy it, and convert to an actual function
    function_code = sanitize_python_code(function_code)
    write_to_py_file(FUNCTIONS_FOLDER, function_name, function_code)
    func = load_function_from_file(FUNCTIONS_FOLDER, function_name)
    return func

In [46]:
func = make_new_function(
    function_name="add_numbers",
    arg_descriptions="numbers: a list of numbers to add up",
    description="Adds up the list of numbers",
)
print(inspect.getsource(func))

def add_numbers(numbers: List[float]) -> float:
    """
    Adds up the list of numbers.

    Args:
        numbers: A list of numbers to add up.

    Returns:
        The sum of the numbers.
    """
    return sum(numbers)



# Integrate the ability to generate new functions


## Build a systemp prompt
We'll make a fairly general system prompt but emphasise that in general it would be good to use functions

In [54]:
SYSTEM_PROMPT_GENERAL = """You are a helpful AI assistant. 
When responding you should follow these rules:
 - You should always use functions as an intermediate step to respond to the user when appropriate (e.g. when being asked to do math, use a function to do the calculation)
 - You should ONLY consider using the functions provided (e.g. do not assume you can use `python`, that does NOT exist)
 - If there is a missing function that would be useful, make a call to `make_new_function` to create it BEFORE responding to the user
"""

GPT-3.5 has a bit of a problem of hallucinating functions that don't exist (e.g. It often assumes it has a `python` function if you let it decide what to do automatically)

## Add the `make_new_function` function to a collection of we'll give to GPT

In [55]:
# Initialize a dictionary that will collect the functions we'll let GPT use
available_functions = {
    "make_new_function": {
        "func": make_new_function,
        "description": get_json_representation(make_new_function),
    }
}


def to_descriptions_list(functions_dict: dict) -> list[dict]:
    """List out the 'description' part of the available functions dict for GPT to use"""
    return [entry["description"] for entry in functions_dict.values()]


# This is what we'll pass to GPT
to_descriptions_list(available_functions)

[{'name': 'make_new_function',
  'description': "Use this if an existing function doesn't exist, and it would be helpful to have a new function to complete a task. The new function will be made to carry out the task described in the `description` given the arguments described by `arg_descriptions`",
  'parameters': {'type': 'object',
   'properties': {'function_name': {'type': 'string'},
    'arg_descriptions': {'type': 'string'},
    'description': {'type': 'string'}},
   'required': ['function_name', 'arg_descriptions', 'description']},
  'returns': {'type': 'callable'}}]

---
So we are starting out with only one function that will be made available to GPT

## Make an example user request

We'll ask for something that GPT is bad at doing by itself (like basic math)

In [56]:
user_request = "What is the result of multiplying together all of these numbers [3,2,6,3,6,5,4,3,6]?"

## Request an initial response
Now we'll see what GPT wants to do with the users request

In [57]:
first_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    temperature=0.0,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT_GENERAL},
        {"role": "user", "content": user_request},
    ],
    functions=to_descriptions_list(available_functions),
    function_call="auto",  # Allow GPT to decide whether to use a function or not
    # function_call={"name": "make_new_function"},  # Force using the specified function
    # function_call='none',  # Force NOT using any functions
)
print(first_response["choices"][0])
print("---------------")
print(
    "Requested Function:\n",
    first_response["choices"][0]["message"]["function_call"]["arguments"],
)

{
  "index": 0,
  "message": {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "make_new_function",
      "arguments": "{\n  \"function_name\": \"multiply_numbers\",\n  \"arg_descriptions\": \"numbers: number[]\",\n  \"description\": \"Multiplies together all the numbers in the given array.\"\n}"
    }
  },
  "finish_reason": "function_call"
}
---------------
Requested Function:
 {
  "function_name": "multiply_numbers",
  "arg_descriptions": "numbers: number[]",
  "description": "Multiplies together all the numbers in the given array."
}


---
Here, GPT is correctly choosing to request a new function. 

Note: If you have difficulty getting GPT to use your function, you can force it to by specifying e.g. `function_call={"name": "make_new_function"}`

## Make the requested function
Now use our `make_new_function` from above to generate the requested function and add it to the list of `available_functions`

In [58]:
# Extract the function_call GPT requested
function_request_message = first_response["choices"][0]["message"]
new_func_arguments = json.loads(function_request_message["function_call"]["arguments"])
new_func_name = new_func_arguments["function_name"]

# Carry out the request
new_func = make_new_function(
    **new_func_arguments,
)

# Add the new function to the list of available functions
available_functions[new_func_name] = {
    "func": new_func,
    "description": get_json_representation(new_func),
}

# Let's see what function we just added
print(f"New function added:\n{inspect.getsource(new_func)}")

New function added:
def multiply_numbers(numbers: List[float]) -> float:
    """
    Multiplies together all the numbers in the given array.

    Args:
        numbers: A list of numbers.

    Returns:
        The product of all the numbers in the list.
    """
    product = 1
    for num in numbers:
        product *= num
    return product



## Request response with updated available functions

In [59]:
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT_GENERAL},
        {"role": "user", "content": user_request},
        # Note: No need to include the message requesting the new func - treat this as the first request again
    ],
    functions=to_descriptions_list(available_functions),
    function_call="auto",
)
print(f"Next response from Chat model:\n{second_response['choices'][0]['message']}")

Next response from Chat model:
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "multiply_numbers",
    "arguments": "{\n  \"numbers\": [3, 2, 6, 3, 6, 5, 4, 3, 6]\n}"
  }
}


---
Excellent, now GPT wants to use the function it just created for itself!

## Get the response from the requested function

In [60]:
# Extract the returned message
function_call_message = second_response["choices"][0]["message"]

# From that message, get the name of the function called
function_name = function_call_message["function_call"]["name"]

# Also get the arguments (this is a string representation of a JSON dict of arguments)
arguments = json.loads(function_call_message["function_call"]["arguments"])

# Make the call to the function with the arguments that GPT decided
function_response = available_functions[function_name]["func"](**arguments)

# Look at what the function returned
print(f"Function output:\n{function_response}")

Function output:
233280


## Convert response into message that GPT understands


In [61]:
function_response_message = {
    "role": "function",
    "name": function_name,
    "content": str(function_response),
}

## Call back to openai including the extra info

In [63]:
final_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT_GENERAL},
        {"role": "user", "content": user_request},
        # Add the context of the previous function call and response
        function_call_message,
        function_response_message,
    ],
)
final_response["choices"][0]["message"]["content"]

'The result of multiplying together all of these numbers [3, 2, 6, 3, 6, 5, 4, 3, 6] is 233280.'

# Make autonomous GPT

This makes the demo shown at the beginning

In [66]:
def print_colored_text(text, color):
    """
    Print colored text in JupyterLab output.
    Available colors: 'red', 'green', 'yellow', 'blue', 'magenta', 'cyan', 'white'
    """
    color_code = {
        "red": "\x1b[31m",
        "green": "\x1b[32m",
        "yellow": "\x1b[33m",
        "blue": "\x1b[34m",
        "magenta": "\x1b[35m",
        "cyan": "\x1b[36m",
        "white": "\x1b[37m",
    }
    reset_code = "\x1b[0m"

    if color not in color_code:
        raise ValueError(
            f"Invalid color. Available colors: {', '.join(color_code.keys())}"
        )

    colored_text = f"{color_code[color]}{text}{reset_code}"
    print(colored_text)


# Example usage
colors = ["red", "green", "yellow", "blue", "magenta", "cyan", "white"]
for color in colors:
    print_colored_text(f"This is {color} text", color)

This is red text
This is green text
This is yellow text
This is blue text
This is magenta text
This is cyan text
This is white text


In [92]:
global_available_functions = {}


def init_available_functions(clean_slate=False):
    global global_available_functions
    if clean_slate is False and global_available_functions:
        return global_available_functions
    else:
        global_available_functions = {
            "make_new_function": {
                "func": make_new_function,
                "description": MAKE_NEW_FUNCTION_DESCRIPTION,
            }
        }
        return global_available_functions


def get_response(messages: list[dict], functions: list[dict] = None) -> dict:
    function_call = "auto" if functions is not None else "none"
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        temperature=0.0,
        messages=messages,
        functions=to_descriptions_list(functions),
        function_call=function_call,
    )


def is_function_call(response: dict) -> bool:
    return response["choices"][0]["finish_reason"] == "function_call"


def get_message(response: dict) -> dict:
    return response["choices"][0]["message"]


def ask_autonomous_gpt(
    question: str, clean_slate=True, verbose=True, max_steps=5
) -> str:
    """
    Ask chatGPT a question, and it will decide to make new functions when necessary to answer your question.
    Args:
        question: The overall question to answer
        clean_slate: Should it remember previously written functions, or start from a clean slate?
        verbose: Have it print messages as it goes (otherwise only returns the final answer)
        max_steps: Max number of times to loop (to prevent getting stuck in a loop and using all your credit!)
    """
    available_functions = init_available_functions(clean_slate=clean_slate)
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT_GENERAL},
        {"role": "user", "content": question},
    ]

    if verbose:
        print_colored_text(f"I've been asked to answer:", "green")
        print_colored_text(question, "white")

    step = 0
    while step <= max_steps:
        step += 1
        response = get_response(messages, available_functions)
        message = get_message(response)
        if is_function_call(response):
            func_call = message["function_call"]
            func_name = func_call["name"]
            func_args = json.loads(func_call["arguments"])
            if func_name == "make_new_function":
                new_func_name = func_args["function_name"]
                if verbose:
                    print_colored_text(
                        f"Hmm, I think I'll first make a function called `{new_func_name}` to help out with this",
                        "green",
                    )
                new_func = make_new_function(**func_args)
                if verbose:
                    print_colored_text(
                        f"I made this function:",
                        "green",
                    )
                    print_colored_text(
                        inspect.getsource(new_func),
                        "magenta",
                    )

                available_functions[new_func_name] = {
                    "func": new_func,
                    "description": get_json_representation(new_func),
                }
                # Note: don't add anything to messages on purpose here (we just pretend the function already existed next time around)
                continue
            else:
                if verbose:
                    print_colored_text(
                        f"I think I should use the `{new_func_name}` function with these arguments:\n{func_args}",
                        "green",
                    )
                function_response = available_functions[func_name]["func"](**func_args)
                if verbose:
                    print_colored_text(
                        f"{func_name} returned: {str(function_response)}", "blue"
                    )
                    print_colored_text(
                        f"I'll keep that in mind as I move forward", "green"
                    )
                function_response_message = {
                    "role": "function",
                    "name": func_name,
                    "content": str(function_response),
                }
                messages.append(message)
                messages.append(function_response_message)
                continue
        else:
            answer = message["content"]
            if verbose:
                if step <= 1:
                    print_colored_text(
                        f"I think I can just answer this question directly:", "green"
                    )
                else:
                    print_colored_text(
                        f"I think I am ready to answer, here goes:", "green"
                    )
                print_colored_text(answer, "cyan")
            return answer
    if verbose:
        print_colored_text(
            f"I have failed to get to the answer soon enough :( You can try increasing max_steps from {max_steps} to a higher number, but the problem may just be too hard for me...",
            "green",
        )
    raise RuntimeError(f"Failed to reach a final answer within {max_steps} steps")

In [91]:
# user_request = "What is the result of multiplying together all of these numbers [3,2,6,3,6,5,4,3,6]?"
# user_request = "How many letters are in this question?"
# user_request = "List out the first 10 fibonnaci numbers? Then what is the product of those 10 numbers? And finally, how many digits are in that answer?"
user_request = "Generate a list of the first 10 fibonnaci numbers starting from 1? Then what is the product of those 10 numbers? And finally, how many digits are in that answer?"
final_answer = ask_autonomous_gpt(user_request, verbose=True, max_steps=10)

I've been asked to answer:
Generate a list of the first 10 fibonnaci numbers starting from 1? Then what is the product of those 10 numbers? And finally, how many digits are in that answer?
{
  "name": "make_new_function",
  "arguments": "{\n  \"function_name\": \"fibonacci_sequence\",\n  \"arg_descriptions\": \"n: number\",\n  \"description\": \"Generates a list of the first n Fibonacci numbers\"\n}"
}
Hmm, I think I'll first make a function called `fibonacci_sequence` to help out with this
I made this function:
def fibonacci_sequence(n: int) -> List[int]:
    """
    Generates a list of the first n Fibonacci numbers

    Args:
        n (int): The number of Fibonacci numbers to generate

    Returns:
        List[int]: A list of the first n Fibonacci numbers
    """
    fib_sequence = [0, 1]
    while len(fib_sequence) < n:
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence[:n]

{
  "name": "fibonacci_sequence",
  "arguments": "{\n  \"n\": 10\n}"


In [80]:
global_available_functions

{'make_new_function': {'func': <function __main__.make_new_function(function_name: str, arg_descriptions: str, description: str) -> Callable>,
  'description': {'name': 'make_new_function',
   'description': "Use this if an existing function doesn't exist, and it would be helpful to have a new function to complete a task. The new function will be made to carry out the task described in the `description` given the arguments described by `arg_descriptions`",
   'parameters': {'type': 'object',
    'properties': {'function_name': {'type': 'string'},
     'arg_descriptions': {'type': 'string'},
     'description': {'type': 'string'}},
    'required': ['function_name', 'arg_descriptions', 'description']},
   'returns': {'type': 'callable'}}},
 'is_prime': {'func': <function temporary.is_prime(n: int) -> bool>,
  'description': {'name': 'is_prime',
   'description': 'Checks if a number is prime',
   'parameters': {'type': 'object',
    'properties': {'n': {'type': 'number',
      'descriptio